In [ ]:
import os
from iqm.qiskit_iqm import IQMProvider
from qiskit.compiler import transpile
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram

In [ ]:
# Set up the Helmi backend
HELMI_CORTEX_URL = os.getenv('HELMI_CORTEX_URL')
if not HELMI_CORTEX_URL:
	raise ValueError("Environment variable HELMI_CORTEX_URL is not set")

provider = IQMProvider(HELMI_CORTEX_URL)
backend = provider.get_backend()

In [ ]:
# Create a bell state: 
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all() #expected equal majority counts of '00' and '11'

qc.draw()

In [ ]:
transpiled_circuit = transpile(qc, backend, optimization_level=3)
transpiled_circuit.draw()

In [ ]:
job = backend.run(transpiled_circuit, shots=1000)
result = job.result()
counts = result.get_counts()

In [ ]:
plot_histogram(counts)